In [1]:
    from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Dataset/train.ft.txt.bz2.zip" -d "/content/Amazon Reviews"

Archive:  /content/drive/MyDrive/Dataset/train.ft.txt.bz2.zip
  inflating: /content/Amazon Reviews/train.ft.txt.bz2  


In [9]:
import bz2 as bz2
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('/content/Amazon Reviews/train.ft.txt.bz2')

In [12]:
from sklearn.model_selection import train_test_split
labels_train,discard_labels,texts_train,discard_texts = train_test_split(train_labels,train_texts,
                                                 test_size = 0.99,
                                                 random_state = 2021)

In [13]:

print(len(train_texts))
print(len(texts_train))
print(len(discard_texts))



3600000
36000
3564000


In [7]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_object = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
snowball_object = SnowballStemmer("english")

In [14]:
reviews_train = []
for text in texts_train:
    text = text.lower() #Convert to lower case
    text = re.sub('[^a-zA-Z ]','',text)
    text = text.split()
    text = [porter_object.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    reviews_train.append(text)

In [15]:
df_to_save_train = pd.DataFrame(reviews_train)


In [16]:
df_to_save_train.to_csv('reviews_train.csv')


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVec = TfidfVectorizer()
x = tfidfVec.fit_transform(reviews_train)

In [18]:
y=labels_train


In [20]:
y = np.reshape(y,(-1,1))
print(y.shape)
print(x.shape)

(36000, 1)
(36000, 71990)


In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size = 0.2,
                                                 random_state = 2021)

In [118]:
from sklearn.svm import SVC
svc_classifier = SVC(C= 0.6, gamma=0.45, kernel='rbf')
svc_classifier.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=0.6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.45, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [119]:
import pickle
pickle.dump(svc_classifier,open('ASA_SVM.sav','wb'))

In [120]:
svc_loaded_classifier = pickle.load(open('/content/ASA_SVM.sav','rb'))

In [121]:
y_pred = svc_loaded_classifier.predict(x_test)

In [122]:
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(y_test,y_pred)*100)

Accuracy:  87.1111111111111


In [123]:
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test,y_pred)
print(c_m)

[[3126  457]
 [ 471 3146]]


In [124]:
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred,pos_label = 0)
print("Out of all the data predicted as negative , what is the accuracy?")
print(prec*100)
from sklearn.metrics import precision_score
prec = precision_score(y_test,y_pred,pos_label = 1)
print("Out of all the data predicted as Positive, what is the accuracy?")
print(prec*100)

Out of all the data predicted as negative , what is the accuracy?
86.90575479566304
Out of all the data predicted as Positive, what is the accuracy?
87.31612545101305


In [125]:
from sklearn.metrics import f1_score 
f1 = f1_score(y_test,y_pred)
print(f1*100)

87.14681440443213


In [126]:
#How to perform gridsearch
from sklearn.model_selection import GridSearchCV
parameters_grid = [{'C':[0.4,0.5,0.6],'kernel':['rbf'],'gamma':[0.4,0.45,0.5,0.55,0.6]}]
grid_search_results = GridSearchCV(estimator = svc_classifier,
                                   param_grid = parameters_grid,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   cv = 10)

In [104]:
a_train,a_test,b_train,b_test = train_test_split(reviews_train,labels_train,
                                                 test_size = 0.8,
                                                 random_state = 2021)

In [105]:
print(len(a_train))
print(len(b_train))

7200
7200
